<a href="https://colab.research.google.com/github/AndreyDyachkov/Data_analytics_with_Python_2/blob/main/Health_insurance_frequency_analysis/Health_insurance_frequency_analysis_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Insurance loss frequency analysis for a health insurance portofolio segment

(All sensitive data were removed)

### Goal

Define the list of health insurance contracts which badly affected the frequency rate in Feb 2023 

### Plan

1. Get the list of key accounts (number of policyholders > 75 percentile) that experienced a 3 percentage point increase in frequency in Feb 2023.
2. Look at contract details for those key accounts.

### Conclusion

The high loss frequency rate in Feb 2023 was due to two groups of clients:
1. A new large unprofitable client: <client> - that has constantly increasing loss rate since the start of the contract.
2. A group of old clients which are connected and faced redundancy at that time.

###### Imports

In [62]:
import pandas as pd

In [63]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/frequency_analysis/data/data_frequency.csv')
df.head()

,HOLDING,MedOrg.Company_name,CONTRCODE,BEGDATE,CloseEndDate,number of policyholders,number of visitors,FrequencyPerMonth,YearMonthnumber
0,group19,Company 25,AB-C037-0000455,06/09/2021 00:00,05/09/2022 00:00,5213.096774,1129.0,"21,7%",2022/03
1,group19,Company 25,AB-C037-0000455,06/09/2021 00:00,05/09/2022 00:00,5192.733333,965.0,"18,6%",2022/04
2,group19,Company 25,AB-C037-0000788,06/09/2022 00:00,05/09/2023 00:00,4947.419355,920.0,"18,6%",2023/01
3,group19,Company 25,AB-C037-0000788,06/09/2022 00:00,05/09/2023 00:00,4977.600000,913.0,"18,3%",2022/11
4,group19,Company 25,AB-C037-0000788,06/09/2022 00:00,05/09/2023 00:00,4942.000000,888.0,"18,0%",2023/02


Rename columns

In [64]:
df.columns=['holding','company','contract_id','start_date','end_date', 'count_policies','count_unique_claims','frequency','year_month']
df.head()

,holding,company,contract_id,start_date,end_date,count_policies,count_unique_claims,frequency,year_month
0,group19,Company 25,AB-C037-0000455,06/09/2021 00:00,05/09/2022 00:00,5213.096774,1129.0,"21,7%",2022/03
1,group19,Company 25,AB-C037-0000455,06/09/2021 00:00,05/09/2022 00:00,5192.733333,965.0,"18,6%",2022/04
2,group19,Company 25,AB-C037-0000788,06/09/2022 00:00,05/09/2023 00:00,4947.419355,920.0,"18,6%",2023/01
3,group19,Company 25,AB-C037-0000788,06/09/2022 00:00,05/09/2023 00:00,4977.600000,913.0,"18,3%",2022/11
4,group19,Company 25,AB-C037-0000788,06/09/2022 00:00,05/09/2023 00:00,4942.000000,888.0,"18,0%",2023/02


###### Data processing

In [65]:
df.shape

(1652, 9)

In [66]:
df.duplicated().sum()

0

In [67]:
df.isnull().mean().sort_values(ascending=False)

count_unique_claims    0.171308
frequency              0.171308
holding                0.159806
count_policies         0.003027
company                0.000000
contract_id            0.000000
start_date             0.000000
end_date               0.000000
year_month             0.000000
dtype: float64

In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1652 entries, 0 to 1651
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   holding              1388 non-null   object 
 1   company              1652 non-null   object 
 2   contract_id          1652 non-null   object 
 3   start_date           1652 non-null   object 
 4   end_date             1652 non-null   object 
 5   count_policies       1647 non-null   float64
 6   count_unique_claims  1369 non-null   float64
 7   frequency            1369 non-null   object 
 8   year_month           1652 non-null   object 
dtypes: float64(2), object(7)
memory usage: 116.3+ KB


In [69]:
df['company'].value_counts()

Company 26     35
Company 27     32
Company 25     26
Company 140    26
Company 145    24
               ..
Company 79      2
Company 45      2
Company 57      2
Company 92      2
Company 38      2
Name: company, Length: 147, dtype: int64

Unify company name

In [70]:
df['company'] = df['company'].str.lower()
df['company'] = df['company'].str.strip()

Replace missing values in the holding column with company names

In [71]:
df['holding'] = df['holding'].fillna(df['company'])

Correct data types

In [72]:
df['start_date'] = pd.to_datetime(df['start_date'])
df['end_date'] = pd.to_datetime(df['end_date'])

Fill missing values with 0 and change data types to integer

In [73]:
df['count_policies'] = df['count_policies'].fillna(0)
df['count_policies'] = df['count_policies'].round(0).astype(int)

In [74]:
df['count_unique_claims'] = df['count_unique_claims'].fillna(0)
df['count_unique_claims'] = df['count_unique_claims'].astype(int)

Change data type for the frequency column from string to float

In [75]:
df['frequency'] = (df['frequency'].str.replace(',','.').str.rstrip('%').astype(float)/100).round(2)

Result of data cleaning

In [76]:
df.head()

,holding,company,contract_id,start_date,end_date,count_policies,count_unique_claims,frequency,year_month
0,group19,company 25,AB-C037-0000455,2021-06-09,2022-05-09,5213,1129,0.22,2022/03
1,group19,company 25,AB-C037-0000455,2021-06-09,2022-05-09,5193,965,0.19,2022/04
2,group19,company 25,AB-C037-0000788,2022-06-09,2023-05-09,4947,920,0.19,2023/01
3,group19,company 25,AB-C037-0000788,2022-06-09,2023-05-09,4978,913,0.18,2022/11
4,group19,company 25,AB-C037-0000788,2022-06-09,2023-05-09,4942,888,0.18,2023/02


EDA

In [77]:
df.describe()

,count_policies,count_unique_claims,frequency
count,1652.000000,1652.000000,1369.000000
mean,275.388015,34.898305,0.265712
std,788.017327,96.893158,0.467211
min,0.000000,0.000000,0.000000
25%,7.000000,1.000000,0.140000
50%,28.000000,4.000000,0.220000
75%,166.500000,25.000000,0.290000
max,6127.000000,1129.000000,15.500000


Let's group by holding, company, and year_month to highlight "bad" companies. We will recalculate the frequency for each company.

In [78]:
df_grouped = df.groupby(['holding','company','year_month'], as_index=False).agg({'count_policies':'sum','count_unique_claims':'sum'})
df_grouped['calculated_frequency'] = (df_grouped['count_unique_claims'] / df_grouped['count_policies']).round(2)
df_grouped

,holding,company,year_month,count_policies,count_unique_claims,calculated_frequency
0,company 10,company 10,2022/03,3483,1,0.0
1,company 10,company 10,2022/04,3483,1,0.0
2,company 10,company 10,2022/05,3483,1,0.0
3,company 10,company 10,2022/06,3483,1,0.0
4,company 10,company 10,2022/07,3483,1,0.0
...,...,...,...,...,...,...
1479,group9,company 87,2022/10,5,1,0.2
1480,group9,company 87,2022/11,5,1,0.2
1481,group9,company 87,2022/12,5,1,0.2
1482,group9,company 87,2023/01,5,2,0.4


Now we will pivot the grouped table to get months in columns 

In [79]:
df_pivoted = df_grouped.pivot_table(index=['holding','company'], values=['calculated_frequency'], aggfunc='mean', columns='year_month')
df_pivoted.columns = df_pivoted.columns.droplevel(0)
df_pivoted.columns.name = None
df_pivoted = df_pivoted.reset_index()
df_pivoted

,holding,company,2022/03,2022/04,2022/05,2022/06,2022/07,2022/08,2022/09,2022/10,2022/11,2022/12,2023/01,2023/02
0,company 10,company 10,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,company 100,company 100,0.00,0.23,0.08,0.00,0.00,0.00,0.17,0.42,0.17,0.08,0.17,0.17
2,company 101,company 101,0.13,0.14,0.15,0.12,0.12,0.12,0.12,0.15,0.15,0.25,NaN,NaN
3,company 107,company 107,0.14,0.14,0.14,0.14,0.21,0.21,0.29,0.00,0.00,0.29,0.15,0.25
4,company 11,company 11,0.33,inf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,group8,company 98,0.20,0.00,0.00,0.00,0.60,0.00,0.50,0.62,0.00,0.22,0.22,0.11
149,group9,company 34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.25,0.25,0.00,0.00,0.00
150,group9,company 62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.33,0.33,0.33,0.00,0.00
151,group9,company 86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.25,0.25,0.25,0.25


Calculating average frequency over all month

In [80]:
df_pivoted['avg_frequency'] = df_pivoted.mean(axis=1, numeric_only=True).round(2)
df_pivoted

,holding,company,2022/03,2022/04,2022/05,2022/06,2022/07,2022/08,2022/09,2022/10,2022/11,2022/12,2023/01,2023/02,avg_frequency
0,company 10,company 10,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,company 100,company 100,0.00,0.23,0.08,0.00,0.00,0.00,0.17,0.42,0.17,0.08,0.17,0.17,0.12
2,company 101,company 101,0.13,0.14,0.15,0.12,0.12,0.12,0.12,0.15,0.15,0.25,NaN,NaN,0.14
3,company 107,company 107,0.14,0.14,0.14,0.14,0.21,0.21,0.29,0.00,0.00,0.29,0.15,0.25,0.16
4,company 11,company 11,0.33,inf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,inf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,group8,company 98,0.20,0.00,0.00,0.00,0.60,0.00,0.50,0.62,0.00,0.22,0.22,0.11,0.21
149,group9,company 34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.25,0.25,0.00,0.00,0.00,0.10
150,group9,company 62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.33,0.33,0.33,0.00,0.00,0.20
151,group9,company 86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.25,0.25,0.25,0.25,0.20


Difference between 2023/02 and avg

In [81]:
df_pivoted['growth'] = df_pivoted['2023/02']-df_pivoted['avg_frequency']
df_pivoted

,holding,company,2022/03,2022/04,2022/05,2022/06,2022/07,2022/08,2022/09,2022/10,2022/11,2022/12,2023/01,2023/02,avg_frequency,growth
0,company 10,company 10,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,company 100,company 100,0.00,0.23,0.08,0.00,0.00,0.00,0.17,0.42,0.17,0.08,0.17,0.17,0.12,0.05
2,company 101,company 101,0.13,0.14,0.15,0.12,0.12,0.12,0.12,0.15,0.15,0.25,NaN,NaN,0.14,NaN
3,company 107,company 107,0.14,0.14,0.14,0.14,0.21,0.21,0.29,0.00,0.00,0.29,0.15,0.25,0.16,0.09
4,company 11,company 11,0.33,inf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,inf,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,group8,company 98,0.20,0.00,0.00,0.00,0.60,0.00,0.50,0.62,0.00,0.22,0.22,0.11,0.21,-0.10
149,group9,company 34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.25,0.25,0.00,0.00,0.00,0.10,-0.10
150,group9,company 62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.33,0.33,0.33,0.00,0.00,0.20,-0.20
151,group9,company 86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.25,0.25,0.25,0.25,0.20,0.05


Filtering by frequency difference: more than 3% (portfolio avg for Feb)

In [82]:
df_pivoted_to_check = df_pivoted[df_pivoted['growth']>=0.03]
df_pivoted_to_check

,holding,company,2022/03,2022/04,2022/05,2022/06,2022/07,2022/08,2022/09,2022/10,2022/11,2022/12,2023/01,2023/02,avg_frequency,growth
1,company 100,company 100,0.00,0.23,0.08,0.00,0.00,0.00,0.17,0.42,0.17,0.08,0.17,0.17,0.12,0.05
3,company 107,company 107,0.14,0.14,0.14,0.14,0.21,0.21,0.29,0.00,0.00,0.29,0.15,0.25,0.16,0.09
7,company 122,company 122,0.00,0.00,0.00,0.00,0.00,0.50,0.25,0.25,0.75,0.00,0.50,0.25,0.21,0.04
8,company 126,company 126,0.18,0.14,0.14,0.14,0.09,0.05,0.05,0.23,0.14,0.14,0.09,0.23,0.14,0.09
9,company 127,company 127,0.03,0.00,0.00,0.00,0.00,0.05,0.06,0.07,0.08,0.07,0.07,0.08,0.04,0.04
10,company 13,company 13,0.03,0.03,0.00,0.02,0.02,0.02,0.03,0.00,0.02,0.05,0.02,0.05,0.02,0.03
11,company 131,company 131,0.14,0.14,0.14,0.00,0.14,0.00,0.20,0.14,0.14,0.14,0.00,0.14,0.11,0.03
14,company 20,company 20,0.35,0.52,0.28,0.27,0.22,0.19,0.19,0.25,0.26,0.31,0.32,0.36,0.29,0.07
16,company 32,company 32,0.18,0.18,0.18,0.09,0.00,0.09,0.00,0.18,0.18,0.00,0.08,0.17,0.11,0.06
19,company 53,company 53,0.12,0.16,0.04,0.00,0.04,0.04,0.16,0.12,0.04,0.08,0.12,0.25,0.10,0.15


Creating a table of key accounts (by the number of policies) to filter the previous table.

In [83]:
df_policies_on_feb = df[(df['year_month']=='2023/02')]
df_policies_on_feb = df_policies_on_feb[['company','count_policies']]
df_policies_on_feb

,company,count_policies
4,company 25,4942
13,company 26,2039
31,company 2,1596
45,company 14,1068
56,company 17,833
...,...,...
1635,company 71,5
1637,company 72,6
1639,company 92,1
1642,company 38,11


In [84]:
df_policies_on_feb.describe()

,count_policies
count,144.000000
mean,256.937500
std,759.247974
min,1.000000
25%,6.750000
50%,25.500000
75%,181.500000
max,6127.000000


In [85]:
df_policies_on_feb = df_policies_on_feb[df_policies_on_feb['count_policies']>182]
df_policies_on_feb

,company,count_policies
4,company 25,4942
13,company 26,2039
31,company 2,1596
45,company 14,1068
56,company 17,833
57,company 39,732
68,company 132,715
79,company 5,581
88,company 69,708
102,company 4,642


### The list of key accounts with the frequency growth that affected the overall portfolio frequency in Feb 2023.

In [86]:
filtered_df = df_pivoted_to_check.merge(df_policies_on_feb, how='inner', left_on='company', right_on='company')
filtered_df.round(2)
filtered_df

,holding,company,2022/03,2022/04,2022/05,2022/06,2022/07,2022/08,2022/09,2022/10,2022/11,2022/12,2023/01,2023/02,avg_frequency,growth,count_policies
0,company 127,company 127,0.03,0.00,0.00,0.00,0.00,0.05,0.06,0.07,0.08,0.07,0.07,0.08,0.04,0.04,1595
1,group12,company 69,0.13,0.14,0.14,0.14,0.15,0.14,0.15,0.17,0.19,0.18,0.16,0.20,0.16,0.04,708
2,group13,company 65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.12,0.20,0.11,0.09,312
3,group16,company 27,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.04,0.03,0.03,0.18,0.21,0.06,0.15,409
4,group20,company 63,0.19,0.23,0.16,0.18,0.11,0.19,0.20,0.18,0.18,0.25,0.17,0.22,0.19,0.03,213
5,group22,company 132,NaN,NaN,NaN,NaN,NaN,NaN,0.09,0.26,0.27,0.25,0.22,0.27,0.23,0.04,715
6,group22,company 5,NaN,NaN,NaN,NaN,NaN,NaN,0.06,0.24,0.23,0.26,0.24,0.29,0.22,0.07,581
7,group6,company 103,0.22,0.25,0.20,0.23,0.19,0.23,0.20,0.22,0.23,0.26,0.18,0.26,0.22,0.04,193
8,group8,company 26,0.17,0.22,0.21,0.19,0.19,0.24,0.22,0.22,0.25,0.21,0.22,0.25,0.22,0.03,2039


##### Creat list of the companies to check their contracts

In [87]:
companies_to_filter = filtered_df['company']
companies_to_filter

0    company 127
1     company 69
2     company 65
3     company 27
4     company 63
5    company 132
6      company 5
7    company 103
8     company 26
Name: company, dtype: object

Get data from the initial table

In [88]:
filter_month = ['2022/10','2022/11','2022/12','2023/01','2023/02']
df_to_check = df[(df['company'].isin(companies_to_filter)) & (df['year_month'].isin(filter_month))]
df_to_check

,holding,company,contract_id,start_date,end_date,count_policies,count_unique_claims,frequency,year_month
12,group8,company 26,AB-C037-0000733,2022-01-07,2023-06-30,2026,520,0.26,2022/11
13,group8,company 26,AB-C037-0000733,2022-01-07,2023-06-30,2039,517,0.25,2023/02
18,group8,company 26,AB-C037-0000733,2022-01-07,2023-06-30,2039,449,0.22,2022/10
19,group8,company 26,AB-C037-0000733,2022-01-07,2023-06-30,2023,436,0.22,2023/01
21,group8,company 26,AB-C037-0000733,2022-01-07,2023-06-30,2041,431,0.21,2022/12
66,group22,company 132,AB-C037-0000853,2022-09-30,2023-09-29,718,194,0.27,2022/11
68,group22,company 132,AB-C037-0000853,2022-09-30,2023-09-29,715,192,0.27,2023/02
75,group22,company 132,AB-C037-0000853,2022-09-30,2023-09-29,706,181,0.26,2022/10
76,group22,company 132,AB-C037-0000853,2022-09-30,2023-09-29,720,180,0.25,2022/12
79,group22,company 5,AB-C037-0000855,2022-09-30,2023-09-29,581,167,0.29,2023/02


### Contract details for "bad" clients

In [89]:
df_to_check_pivoted = df_to_check.pivot_table(index=['holding','company','contract_id','start_date','end_date'], values=['count_policies','count_unique_claims','frequency'],  aggfunc=sum, columns='year_month')
#df_to_check_pivoted.columns = df_to_check_pivoted.columns.droplevel(0)
#df_to_check_pivoted.columns.name = None
#df_to_check_pivoted = df_to_check_pivoted.reset_index()
df_to_check_pivoted

count_policies  \
year_month                                                           2022/10   
holding     company     contract_id     start_date end_date                    
company 127 company 127 AB-C037-0000477 2021-01-09 2022-12-31         1478.0   
                        AB-C037-0000980 2023-01-01 2023-12-31            NaN   
group12     company 69  AB-C037-0000653 2022-02-27 2023-02-26          750.0   
group13     company 65  AB-C037-0000973 2022-12-30 2023-12-29            NaN   
group16     company 27  AB-C037-0000058 2020-01-03 2022-12-31          437.0   
                        AB-C037-0000111 2020-01-05 2022-12-31         2828.0   
                        AB-C037-0000252 2020-10-15 2022-12-31          130.0   
                        AB-C037-0000976 2023-01-01 2023-12-31            NaN   
group20     company 63  AB-C037-0000540 2021-01-11 2022-12-31          241.0   
                        AB-C037-0000949 2022-12-31 2023-12-30            NaN   
group22     company 132 AB-C037-0000853 2022-09-30 2023-09-29          706.0   
            company 5   AB-C037-0000855 2022-09-30 2023-09-29          591.0   
group6      company 103 AB-C037-0000804 2022-10-09 2023-09-09          190.0   
            company 69  AB-C037-0001042 2023-02-27 2024-02-26            NaN   
group8      company 26  AB-C037-0000209 2020-01-11 2023-01-16           35.0   
                        AB-C037-0000480 2021-02-10 2022-01-10            0.0   
                        AB-C037-0000733 2022-01-07 2023-06-30         2039.0   
                        AB-C037-0000819 2022-02-10 2022-12-31            1.0   

                                                                               \
year_month                                                    2022/11 2022/12   
holding     company     contract_id     start_date end_date                     
company 127 company 127 AB-C037-0000477 2021-01-09 2022-12-31  1472.0  1478.0   
                        AB-C037-0000980 2023-01-01 2023-12-31     NaN     0.0   
group12     company 69  AB-C037-0000653 2022-02-27 2023-02-26   752.0   761.0   
group13     company 65  AB-C037-0000973 2022-12-30 2023-12-29     NaN    20.0   
group16     company 27  AB-C037-0000058 2020-01-03 2022-12-31   404.0   405.0   
                        AB-C037-0000111 2020-01-05 2022-12-31  2828.0  2828.0   
                        AB-C037-0000252 2020-10-15 2022-12-31   130.0   130.0   
                        AB-C037-0000976 2023-01-01 2023-12-31     NaN     NaN   
group20     company 63  AB-C037-0000540 2021-01-11 2022-12-31   232.0   232.0   
                        AB-C037-0000949 2022-12-31 2023-12-30     NaN     6.0   
group22     company 132 AB-C037-0000853 2022-09-30 2023-09-29   718.0   720.0   
            company 5   AB-C037-0000855 2022-09-30 2023-09-29   611.0   601.0   
group6      company 103 AB-C037-0000804 2022-10-09 2023-09-09   190.0   191.0   
            company 69  AB-C037-0001042 2023-02-27 2024-02-26     NaN     NaN   
group8      company 26  AB-C037-0000209 2020-01-11 2023-01-16    32.0    33.0   
                        AB-C037-0000480 2021-02-10 2022-01-10     NaN     NaN   
                        AB-C037-0000733 2022-01-07 2023-06-30  2026.0  2041.0   
                        AB-C037-0000819 2022-02-10 2022-12-31     1.0     1.0   

                                                                               \
year_month                                                    2023/01 2023/02   
holding     company     contract_id     start_date end_date                     
company 127 company 127 AB-C037-0000477 2021-01-09 2022-12-31     NaN     NaN   
                        AB-C037-0000980 2023-01-01 2023-12-31  1576.0  1595.0   
group12     company 69  AB-C037-0000653 2022-02-27 2023-02-26   766.0   708.0   
group13     company 65  AB-C037-0000973 2022-12-30 2023-12-29   312.0   312.0   
group16     company 27  AB-C037-0000058 2020-01-03 2022-12-31     NaN     NaN   
                        AB-C037